# Scratch Blocks in Haskell

## Scratch ↔ Haskell Mappings

| Scratch Block | Haskell Equivalent | Notes |
|---------------|-------------------|-------|
| **Variables** | `IORef` | Mutable state in IO monad |
| `set my variable to 0` | `writeIORef myVariable 0` | Set variable value |
| `change my variable by 1` | `modifyIORef myVariable (+1)` | Modify variable |
| `my variable` | `readIORef myVariable` | Read variable value |
| **Control Flow** | | |
| `forever` | `forever` from `Control.Monad` | Infinite loop |
| `repeat until <condition>` | `repeatUntil` (custom) | Loop until condition true |
| `if <condition> then` | `when` from `Control.Monad` | Conditional execution |
| `wait 1 seconds` | `threadDelay 1000000` | Delay (in microseconds) |
| **Random & Math** | | |
| `pick random 1 to 10` | `randomIO 1 10` (custom) or `randomRIO (1, 10)` | Random number generation |
| **Motion** | | |
| `goto x:0 y:0` | `gotoXY 0 0` (custom function) | Set position |
| **Sensing** | | |
| `touching color black?` | `touchingColorBlack` (simulated) | Boolean check |
| **Events** | | |
| `when green flag clicked` | `main :: IO ()` | Program entry point |
| `when right arrow key pressed` | Event loop with `getLine` | Keyboard input (getChar doesn't work in IHaskell) |
| `when this sprite clicked` | Function call | Simulated event handler |

## Required Imports

**Note:** You may need to install the `random` package first if you want to use `randomRIO`:

```bash
# In your terminal (not in IHaskell):
cabal install --lib random
```

However, the notebook provides a `randomIO` function that works without any extra packages.

```haskell
import Control.Monad (forever, when, unless)
import Control.Concurrent (threadDelay)
import Data.IORef
import Data.Time.Clock.POSIX (getPOSIXTime)  -- For randomIO
```

**IHaskell Limitations:**
- `getChar` doesn't work in IHaskell (causes "end of file" error) - use `getLine` instead
- Interactive stdin is limited - better to run interactive programs outside Jupyter

## Note on System.Random

If you get a "Could not find module 'System.Random'" error, you have two options:

1. **Install random package outside IHaskell:**
   ```bash
   # In your terminal (not in this notebook):
   cabal install --lib random
   # or with stack:
   stack install random
   ```

2. **Use the alternative `simpleRandom` function provided below** (no installation needed)

## Setup and Imports

In [2]:
import Control.Monad (forever, when, unless)
import Control.Concurrent (threadDelay)
import Data.IORef

-- Note: System.Random is optional - we provide randomIO below
-- import System.Random (randomRIO)

### Random Number Generator (no extra packages needed)

In [3]:
-- Simple pseudo-random number generator using system time
-- This works without needing System.Random package
import Data.Time.Clock.POSIX (getPOSIXTime)

randomIO :: Int -> Int -> IO Int
randomIO low high = do
    time <- getPOSIXTime
    let seed = round (time * 1000000) :: Int
    return $ low + (seed `mod` (high - low + 1))

-- Test it
randomIO 1 10

7

## Define Variables

In [4]:
-- Create mutable variables (like Scratch variables)
counter <- newIORef (0 :: Int)
myVariable <- newIORef (0 :: Int)

-- Position variables
xPos <- newIORef (0 :: Int)
yPos <- newIORef (0 :: Int)

## Set my variable to 0

In [5]:
-- set my variable to 0
writeIORef myVariable 0

-- Check the value
readIORef myVariable

0

## Change my variable by 1

In [6]:
-- change my variable by 1
modifyIORef myVariable (+1)

-- Check the value
readIORef myVariable

1

## Pick random 1 to 10

In [7]:
-- pick random 1 to 10
randomValue <- randomIO 1 10
putStrLn $ "Random value: " ++ show randomValue

-- Or assign to a variable
writeIORef myVariable =<< randomIO 1 10
readIORef myVariable

Random value: 9

1

## Goto x:0 y:0

In [8]:
-- goto x:0 y:0
gotoXY :: Int -> Int -> IO ()
gotoXY x y = do
    writeIORef xPos x
    writeIORef yPos y
    putStrLn $ "Moved to (" ++ show x ++ ", " ++ show y ++ ")"

-- Execute
gotoXY 0 0

Moved to (0, 0)

## If then

In [9]:
-- if <condition> then
ifThen :: IO ()
ifThen = do
    val <- readIORef myVariable
    -- if my variable > 5 then say "Big number!"
    when (val > 5) $ do
        putStrLn "Big number!"

-- Test it
writeIORef myVariable 7
ifThen

writeIORef myVariable 3
ifThen

Big number!

## Repeat until

In [10]:
-- repeat until <condition>
repeatUntil :: IO Bool -> IO () -> IO ()
repeatUntil condition action = do
    action
    done <- condition
    unless done $ repeatUntil condition action

-- Example: repeat until counter = 5
repeatUntilExample :: IO ()
repeatUntilExample = do
    writeIORef counter 0
    repeatUntil checkCondition $ do
        count <- readIORef counter
        putStrLn $ "Counter: " ++ show count
        modifyIORef counter (+1)
        threadDelay 500000  -- 0.5 seconds
  where
    checkCondition = do
        count <- readIORef counter
        return (count >= 5)

-- Run it
repeatUntilExample

Counter: 0
Counter: 1
Counter: 2
Counter: 3
Counter: 4

## Touching color black? (simulated)

In [11]:
-- touching color black?
-- In a real game engine, this would check pixel collision
-- Here we simulate it with a boolean variable

isTouchingBlack <- newIORef False

touchingColorBlack :: IO Bool
touchingColorBlack = readIORef isTouchingBlack

-- Test it
writeIORef isTouchingBlack True
result <- touchingColorBlack
putStrLn $ "Touching black? " ++ show result

Touching black? True

## When Green Flag clicked (program start)

In [12]:
-- when green flag clicked (main program entry point)
whenGreenFlagClicked :: IO ()
whenGreenFlagClicked = do
    putStrLn "Green flag clicked! Starting program..."
    writeIORef counter 0
    writeIORef myVariable 0
    gotoXY 0 0
    putStrLn "Initialization complete!"

-- Run it
whenGreenFlagClicked

Green flag clicked! Starting program...
Moved to (0, 0)
Initialization complete!

## When right arrow key pressed (event handling)

**Note:** `getChar` doesn't work in IHaskell. Here's a workaround using `getLine`:

In [13]:
-- when right arrow key pressed
-- IHaskell workaround: use getLine instead of getChar

whenRightArrowPressed :: IO ()
whenRightArrowPressed = do
    putStrLn "Type 'r' for right arrow, 'l' for left, 'q' to quit (then press Enter):"
    checkKey
  where
    checkKey = do
        input <- getLine
        case input of
            "r" -> do
                x <- readIORef xPos
                writeIORef xPos (x + 10)
                newX <- readIORef xPos
                putStrLn $ "Moved right! New x: " ++ show newX
                checkKey
            "l" -> do
                x <- readIORef xPos
                writeIORef xPos (x - 10)
                newX <- readIORef xPos
                putStrLn $ "Moved left! New x: " ++ show newX
                checkKey
            "q" -> putStrLn "Quitting..."
            _   -> do
                putStrLn "Invalid input. Use 'r', 'l', or 'q'"
                checkKey

-- Run it
whenRightArrowPressed

: 

## When this sprite clicked (event handler)

In [14]:
-- when this sprite clicked
-- Simulated as a function that can be called
whenSpriteClicked :: IO ()
whenSpriteClicked = do
    putStrLn "Sprite was clicked!"
    -- Do something when clicked
    modifyIORef myVariable (+1)
    val <- readIORef myVariable
    putStrLn $ "Click count: " ++ show val

-- Simulate clicking the sprite
whenSpriteClicked
whenSpriteClicked
whenSpriteClicked

Sprite was clicked!
Click count: 1

Sprite was clicked!
Click count: 2

Sprite was clicked!
Click count: 3

## Forever loop example

In [15]:
-- forever loop (WARNING: runs infinitely!)
-- Use Kernel -> Interrupt to stop

loopForever :: IO ()
loopForever = forever $ do
    count <- readIORef counter
    putStrLn $ "hello " ++ show count
    modifyIORef counter (+1)
    threadDelay 1000000  -- 1 second

-- Uncomment to run (be ready to interrupt!):
-- loopForever

## Limited loop for testing

In [16]:
import Control.Monad (replicateM_)

-- Run only 5 iterations for testing
testLoop :: IO ()
testLoop = do
    writeIORef counter 0
    replicateM_ 5 $ do
        count <- readIORef counter
        putStrLn $ "hello " ++ show count
        modifyIORef counter (+1)
        threadDelay 500000  -- 0.5 seconds

testLoop

hello 0
hello 1
hello 2
hello 3
hello 4

## Complete Example: Combining Multiple Blocks

In [17]:
-- A complete program combining several blocks
completeExample :: IO ()
completeExample = do
    putStrLn "=== Starting Complete Example ==="
    
    -- when green flag clicked
    writeIORef myVariable 0
    gotoXY 0 0
    
    -- repeat until my variable = 3
    repeatUntil checkDone $ do
        -- pick random 1 to 10
        randomVal <- randomIO 1 10
        putStrLn $ "Random: " ++ show randomVal
        
        -- if random > 5 then
        when (randomVal > 5) $ do
            putStrLn "  Big number!"
            -- change my variable by 1
            modifyIORef myVariable (+1)
        
        val <- readIORef myVariable
        putStrLn $ "  My variable: " ++ show val
        threadDelay 500000
        putStrLn "=== Example Complete ==="
  where
    checkDone = do
        val <- readIORef myVariable
        return (val >= 3)
    
    

-- Run the complete example
completeExample

=== Starting Complete Example ===
Moved to (0, 0)
Random: 2
  My variable: 0
=== Example Complete ===
Random: 7
  Big number!
  My variable: 1
=== Example Complete ===
Random: 3
  My variable: 1
=== Example Complete ===
Random: 4
  My variable: 1
=== Example Complete ===
Random: 10
  Big number!
  My variable: 2
=== Example Complete ===
Random: 3
  My variable: 2
=== Example Complete ===
Random: 9
  Big number!
  My variable: 3
=== Example Complete ===

## Running a Fully Interactive Program

IHaskell has limitations with interactive input. For a full interactive experience, we've created a standalone Haskell program.

### Download the Interactive Program

The interactive program `interactive_scratch.hs` includes:
- Real-time keyboard input (w/a/s/d for movement)
- Interactive game loop
- Automated demos
- All Scratch blocks working properly

### Option 1: Download from IHaskell

You can download the file directly from this notebook:

In [ ]:
-- This will create the interactive_scratch.hs file in your current directory
:!echo "File should be downloaded via the notebook interface"

-- To see the current directory:
:!pwd

### Option 2: Copy the Code Manually

1. Create a new file called `interactive_scratch.hs`
2. Copy the code from the standalone file (provided separately)
3. Save it in your working directory

### Running the Program

#### Method 1: Compile and Run (Faster)

Open a terminal and run:

```bash
# Compile the program
ghc interactive_scratch.hs

# Run it
./interactive_scratch
```

#### Method 2: Run Directly with runhaskell (Easier)

```bash
runhaskell interactive_scratch.hs
```

#### Method 3: Run from IHaskell Console (Limited)

⚠️ **Note:** This won't work perfectly in Jupyter due to stdin limitations, but you can try:

```haskell
:!runhaskell interactive_scratch.hs
```

### What the Program Does

When you run it, you'll see:

```
==========================================
  Scratch-like Interactive Program
==========================================

Choose a mode:
1. Run automated demos
2. Interactive game loop
Enter choice (1 or 2):
```

**Mode 1 - Automated Demos:**
- Demo 1: Repeat until score reaches 5
- Demo 2: Move in a square pattern
- Demo 3: Forever loop (limited iterations)

**Mode 2 - Interactive Game:**
- `w/a/s/d` - Move up/left/down/right
- `r` - Random position
- `+/-` - Change score
- `0` - Reset to origin
- `q` - Quit

### Features Demonstrated

✅ Real `getChar` for single-key input
✅ `forever` loops (limited in demos)
✅ `repeat until` loops
✅ Random number generation with `System.Random`
✅ Mutable variables with `IORef`
✅ All Scratch blocks working properly

## IHaskell Limitations and Workarounds

### Limitations

IHaskell (Jupyter with Haskell kernel) has some limitations compared to running Haskell programs in a terminal:

1. **No interactive stdin with `getChar`**
   - **Problem:** `getChar` causes "end of file" errors
   - **Workaround:** Use `getLine` instead and read full strings
   - **Example:**
     ```haskell
     -- Don't use:
     c <- getChar
     
     -- Use instead:
     input <- getLine
     let c = head input  -- if you need first character
     ```

2. **Package installation conflicts**
   - **Problem:** `cabal install` inside IHaskell fails due to `GHC_PACKAGE_PATH` conflicts
   - **Workaround:** Install packages in your terminal before starting Jupyter:
     ```bash
     cabal install --lib random
     # Then start jupyter
     jupyter notebook
     ```
   - **Alternative:** Use custom functions that don't require external packages (like our `randomIO`)

3. **Limited real-time interaction**
   - **Problem:** True event loops and real-time keyboard handling don't work well
   - **Workaround:** For interactive programs, export your code to a `.hs` file and run in terminal:
     ```haskell
     -- Save your code to a file, then in terminal:
     ghc myprogram.hs
     ./myprogram
     ```

4. **Infinite loops**
   - **Problem:** `forever` loops will hang the notebook
   - **Workaround:** Use `replicateM_` for testing with a limited number of iterations:
     ```haskell
     import Control.Monad (replicateM_)
     
     -- Instead of: forever $ action
     -- Use for testing:
     replicateM_ 10 $ action  -- runs 10 times
     ```
   - **Note:** You can interrupt infinite loops with Kernel → Interrupt in Jupyter

### Best Practices for IHaskell

✅ **Good for IHaskell:**
- Learning Haskell syntax and concepts
- Testing pure functions
- Data manipulation and analysis
- Mathematical computations
- Simple IO operations with `putStrLn` and `getLine`

❌ **Better in terminal:**
- Interactive games or real-time applications
- Programs with complex user input
- Infinite loops that need to run continuously
- Programs that need unbuffered character input

### Converting IHaskell Code to Standalone Programs

To run your code outside IHaskell:

1. Copy your code into a `.hs` file
2. Add `main :: IO ()` as the entry point
3. Add necessary imports at the top
4. Compile and run:

```haskell
-- myprogram.hs
import Control.Monad
import Data.IORef

main :: IO ()
main = do
    putStrLn "Hello from standalone Haskell!"
    -- Your code here
```

```bash
# Compile
ghc myprogram.hs

# Run
./myprogram
```

Or use `runhaskell` without compiling:
```bash
runhaskell myprogram.hs
```